In [3]:
import csv
import boto3
import pandas

In [ ]:
#Conectando na aws no servico comprehend
comprehend = boto3.client(
    service_name='comprehend',
    region_name='us-east-1',
    aws_access_key_id='xxxxxxxxxx',
    aws_secret_access_key='xxxxxxxxxx')

In [ ]:
#Abrindo arquivo com dados dos tweets minerados
out = pandas.read_csv('output_got.csv')

In [ ]:
#Iterando em cada tweet e passando pela analise de sentimento adicionando uma coluna com o resultado
for i in out.iterrows():
    i[1]['sentiment'] = comprehend.detect_sentiment(Text=i[1]['text'], LanguageCode='pt')['Sentiment']

In [ ]:
#salvando o arquivo
out.to_csv(r'tweets_analytics.csv',index=False, sep=',', header=True)

In [5]:
#Conectando no servico glue da aws
glue = boto3.client(
    service_name='glue',
    region_name='us-east-1',
    aws_access_key_id='xxxxxxxxxx',
    aws_secret_access_key='xxxxxxxxxx')

In [8]:
#Criando database
response = glue.create_database(
    DatabaseInput={
        'Name': 'Rebranding_Analysis',
        'Description': 'Database to analysis rebranding impacts'          
        }
)

{'ResponseMetadata': {'RequestId': '82c8c790-f2a6-11e9-9134-dd719b0489d1',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Sat, 19 Oct 2019 19:27:38 GMT',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '2',
   'connection': 'keep-alive',
   'x-amzn-requestid': '82c8c790-f2a6-11e9-9134-dd719b0489d1'},
  'RetryAttempts': 0}}

In [15]:
#Criando tabela analitica dos tweets
response = glue.create_table(
    DatabaseName='Rebranding_Analysis',
    TableInput={
        'Name': 'Twitter_Mentions',
        'Description': 'string',        
        'StorageDescriptor': {
            'Columns': [
                {
                    'Name': 'date',
                    'Type': 'string'
                },
                {
                    'Name': 'username',
                    'Type': 'string'    
                },
                {
                    'Name': 'sentiment',
                    'Type': 'string'    
                },
                {
                    'Name': 'to',
                    'Type': 'string'    
                },
                {
                    'Name': 'replies',
                    'Type': 'string'    
                },
                {
                    'Name': 'retweets',
                    'Type': 'string'    
                },
                {
                    'Name': 'favorites',
                    'Type': 'string'    
                },
                {
                    'Name': 'text',
                    'Type': 'string'    
                },
                {
                    'Name': 'geo',
                    'Type': 'string'    
                },
                {
                    'Name': 'hashtags',
                    'Type': 'string'    
                },
                {
                    'Name': 'id',
                    'Type': 'string'    
                },
                {
                    'Name': 'permalink',
                    'Type': 'string'    
                } 
            ],
            'Location': 's3://pucminas-tcc/Tweets-analytics/',
            'InputFormat': 'org.apache.hadoop.mapred.TextInputFormat',
            'OutputFormat': 'org.apache.hadoop.hive.ql.io.HiveIgnoreKeyTextOutputFormat',
            'SerdeInfo': {
                'SerializationLibrary': 'org.apache.hadoop.hive.serde2.lazy.LazySimpleSerDe',
                'Parameters': {
                    'field.delim': ','
                }
            },   
            'Compressed': False
        },       
        'Parameters': {
            'skip.header.line.count': '1'
        }
    }
)

{'ResponseMetadata': {'RequestId': 'b98529a9-f2a8-11e9-b9aa-dfb8dcdde954',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Sat, 19 Oct 2019 19:43:29 GMT',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '2',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'b98529a9-f2a8-11e9-b9aa-dfb8dcdde954'},
  'RetryAttempts': 0}}

In [ ]:
#Criando tabela para o wordcloud
response = glue.create_table(
    DatabaseName='Rebranding_Analysis',
    TableInput={
        'Name': 'Words',
        'Description': 'string',        
        'StorageDescriptor': {
            'Columns': [
                {
                    'Name': 'Words',
                    'Type': 'string'
                }
            ],
            'Location': 's3://pucminas-tcc/Wordcloud/',
            'InputFormat': 'org.apache.hadoop.mapred.TextInputFormat',
            'OutputFormat': 'org.apache.hadoop.hive.ql.io.HiveIgnoreKeyTextOutputFormat',
            'SerdeInfo': {
                'SerializationLibrary': 'org.apache.hadoop.hive.serde2.lazy.LazySimpleSerDe',
                'Parameters': {
                    'field.delim': ','
                }
            },   
            'Compressed': False
        },       
        'Parameters': {
            'skip.header.line.count': '1'
        }
    }
)

In [7]:
response = glue.get_tables(
    DatabaseName='twitter-social'    
)
response

{'TableList': [{'Name': 'raw_twitter_social_raw',
   'DatabaseName': 'twitter-social',
   'Owner': 'owner',
   'CreateTime': datetime.datetime(2019, 10, 10, 17, 7, 48, tzinfo=tzlocal()),
   'UpdateTime': datetime.datetime(2019, 10, 18, 15, 23, 12, tzinfo=tzlocal()),
   'LastAccessTime': datetime.datetime(2019, 10, 10, 17, 7, 48, tzinfo=tzlocal()),
   'Retention': 0,
   'StorageDescriptor': {'Columns': [{'Name': 'date', 'Type': 'string'},
     {'Name': 'username', 'Type': 'string'},
     {'Name': 'sentiment', 'Type': 'string'},
     {'Name': 'to', 'Type': 'string'},
     {'Name': 'replies', 'Type': 'bigint'},
     {'Name': 'retweets', 'Type': 'bigint'},
     {'Name': 'favorites', 'Type': 'bigint'},
     {'Name': 'text', 'Type': 'string'},
     {'Name': 'geo', 'Type': 'string'},
     {'Name': 'mentions', 'Type': 'string'},
     {'Name': 'hashtags', 'Type': 'string'},
     {'Name': 'id', 'Type': 'bigint'},
     {'Name': 'permalink', 'Type': 'string'}],
    'Location': 's3://twitter-social

In [17]:
#Conectando no servico de armazenamento s3
s3 = boto3.client(
    service_name='s3',
    region_name='us-east-1',
    aws_access_key_id='xxxxxxxxxx',
    aws_secret_access_key='xxxxxxxxxx')

In [20]:
#fazendo upload do arquivo para o s3
fileName = 'tweets_analytics.csv'
s3.upload_file(fileName,'pucminas-tcc','{}/{}'.format('Tweets-analytics',fileName)) 

In [ ]:
#Iniciando operacao para geracao do wordcloud
from nltk.corpus import stopwords
from nltk import word_tokenize
from collections import Counter
from string import punctuation

In [ ]:
#Criando lista com palavras de ligacao para serem removidas e pontuacoes
stoplist = set(stopwords.words('portuguese') + list(punctuation))

In [ ]:
#Sbrindo arquivo
data = pandas.read_csv('output_got.csv')

In [ ]:
#Realizando tratamento do texto para nao ter problemas na tokenizacao
texts = data['text'].str.lower()
texts = texts.str.replace('[^A-z ]','').str.replace(' +',' ').str.strip()

In [ ]:
#Separando o texto em palavras
words = word_tokenize(str(texts))

In [ ]:
#Realizando filtro de ligacoes e pontuacoes
wordsFiltered = []

for w in words:
    if w not in stoplist:
        wordsFiltered.append(w)

In [ ]:
#Realizando analise de contagem
words_count = Counter(wordsFiltered)
words_count

In [ ]:
#Criando dataframe para analise
df = pd.DataFrame(wordsFiltered,columns =['Words'])
df

In [ ]:
#Exportando dataframe para csv
df.to_csv(r'words.csv',index=False, sep=',')

In [ ]:
#Realizando upload para o S3 da aws
fileName = 'words.csv'
s3.upload_file(fileName,'pucminas-tcc','{}/{}'.format('Wordcloud',fileName)) 